In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
import os
import pandas as pd
#Functional coding
import functools
from functools import partial

In [2]:
'a' in ['a','b','c']

True

In [ ]:
default_settings = {                
    "cap": tf.constant(80), 
    "outside_resist" : tf.constant(4),
    "wall_resist" : tf.constant(1.5),
    "cap_air" : tf.constant(1.006), 
    "cost_air" : tf.constant(1), 
    "time_delta" : tf.constant(1),
    "temp_air" : tf.constant(40),
    "temp_up" : tf.constant(23.5),
    "temp_low" : tf.constant(20.0),
    "temp_outside" : tf.constant(6),
    "temp_hall" : tf.constant(10),
    "penalty" : tf.constant(20000),
    "air_max" : tf.constant(10)
   }

In [ ]:
class HVAC(object):
    def __init__(self, 
                 temp, #STATE
                 air, #ACTION
                 adj_outside, #Adjacent to outside 
                 adj_hall, #Adjacent to hall
                 adj, #Adjacent between rooms
                 rooms, #Room names
                 default_settings):
        self.__dict__.update(default_settings)
        self.temp = temp
        self.air = air
        self.adj_outside = adj_outside
        self.adj_hall = adj_hall
        self.adj = adj
        self.room = room
        
    def ADJ(self, space1, space2):
        for pair in self.adj:
            if space1 in pair and space2 in pair:
                return True
            else:
                return False
                 
    def ADJ_OUTSIDE(self,  space):
        if space in self.adj_outside:
            return True
        else:
            return False
            
    def ADJ_HALL(self, space):
        if space in self.adj_hall:
            return True
        else:
            return False  
        
    def R_OUTSIDE(self, space):
        return self.outside_resist
    
    def R_WALL(self, space1, space2):
        return self.wall_resist
    
    def IS_ROOM(self, space):
        if space in self.rooms:
            return 1.0
        else:
            return 0.0
        
    def CAP(self, space):
        return self.cap
    
    def CAP_AIR(self):
        return self.cap_air
    
    def COST_AIR(self):
        return self.cost_air
    
    def TIME_DELTA(self):
        return self.time_delta
    
    def TEMP_AIR(self):
        return self.temp_air
    
    def TEMP_UP(self, space):
        return self.temp_up
    
    def TEMP_LOW(self, space):
        return self.temp_low
    
    def TEMP_OUTSIDE(self, space):
        return self.temp_outside
    
    def TEMP_HALL(self, space):
        return self.temp_hall
    
    def PENALTY(self):
        return self.penalty
    
    def AIR_MAX(self, space):
        return self.air_max
    
    # Single state function, need map to matrix later
    def _transition(self, space, states, actions):
        previous_state = states[space]
        heating_info = self.TIME_DELTA()/self.CAP(space)*(actions[space]*self.CAP_AIR()*(self.TEMP_AIR()-previous_state)*self.IS_ROOM(space))
        neighbor_info = tf.constant(0)
        for p in self.rooms:
            if self.ADJ(space,p):
                neighbor_info += (states[p]-previous_state)/self.R_WALL(space,p)
        outside_info = tf.constant(0)
        if self.ADJ_OUTSIDE(space):
            outside_info=(self.TEMP_OUTSIDE(space)-previous_state)/self.R_OUTSIDE(space)
        wall_info = tf.constant(0)
        if self.ADJ_WALL(space):
            wall_info=(self.TEMP_HALL(space)-previous_state)/self.self.R_HALL(space)
            
        new_state = previous_state + heating_info + neighbor_info + outside_info + wall_info
        return new_state
            
                